## Multi - Agent Workflow with LangChain 

This project implement a system of three agents using Langchain library. It'll simulate how a e-learning company could automate suport ticket process for students


Lnagchain defines modular components such as `chains`,`tools` and `agents`

There will be agents, each for a specific role.

- 1. Processes and clasifies the user query

- 2. Searches a simulated database for relevant content

- 3. Generates a personalized response for the user



#### 0. Enviroment setup

In [1]:
! pip freeze > ../../requirements.txt



Langchain will orchestrate the agent logic

In [2]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate



In [ ]:
import openai 
import os 
import pandas as pd

API_KEY = os.getenv(API_KEY)
openai.api_key = API_KEY

llm = OpenAI(temperature = 0.5)


#### 2. Simulated database

The dataframe represents the internal content repository. Each record includes a course, topic, lesson, exercise, difficulty and link.



In [8]:
data = {
    "course": ["Basic Deep Learning", "Python for AI", "Computer Vision", "Natural Language Processing"],
    "topic": ["Neural Networks", "Python Programming", "Image Classification", "Text Analysis"],
    "lesson": ["Introduction to Neural Networks", "Loops and Functions", "Basic CNNs", "Embeddings and Transformers"],
    "exercise": ["Implement a simple network", "Create a factorial function", "Classify cats and dogs", "Translate a text using embeddings"],
    "level": ["Intermediate", "Beginner", "Intermediate", "Advanced"],
    "link": [
        "https://eduflowtech.com/deep-learning/redes",
        "https://eduflowtech.com/python/bucles",
        "https://eduflowtech.com/vision/cnn",
        "https://eduflowtech.com/nlp/embeddings"
    ]
}


In [9]:
df = pd.DataFrame(data)
df.head()

,course,topic,lesson,exercise,level,link
0,Basic Deep Learning,Neural Networks,Introduction to Neural Networks,Implement a simple network,Intermediate,https://eduflowtech.com/deep-learning/redes
1,Python for AI,Python Programming,Loops and Functions,Create a factorial function,Beginner,https://eduflowtech.com/python/bucles
2,Computer Vision,Image Classification,Basic CNNs,Classify cats and dogs,Intermediate,https://eduflowtech.com/vision/cnn
3,Natural Language Processing,Text Analysis,Embeddings and Transformers,Translate a text using embeddings,Advanced,https://eduflowtech.com/nlp/embeddings


###  Agent 1:   Query Preprocessor 

This agent interprets intent of the user's. It classifies each query into one of three categories:

    - lesson
    - exercise
    - technical error


In [ ]:
from langchain import LLMChain

prompt_query = PromptTemplate(
    input_variables=["query"],
    template="""
You are an assistant that classifies student queries.
Classify the following query into one of the categories:
- 'lesson'
- 'exercise'
- 'technical error'

Query: {query}

Category:
"""
)

processor_agent = LLMChain(llm=llm, prompt=prompt_query)


###  Agent 2.  Content Searcher 

In [ ]:
def search_content(category,query):
    """
    Searches the DataFrame for the most relevant content 
    according to the detected category and user intent.
    """
    query = query.lower()
    
    if "neural" in query or "network" in query:
        return df[df["topic"].str.contains("Neural", case=False)]
    elif "python" in query:
        return df[df["topic"].str.contains("Python", case=False)]
    elif "image" in query:
        return df[df["topic"].str.contains("Vision", case=False)]
    elif "text" in query or "language" in query:
        return df[df["topic"].str.contains("Language", case=False)]
    else:
        # if nothing matches, return a random row
        return df.sample(1)
    



In [ ]:
# Langchain tool 
tool_search = Tool(

    name = 'ContentSearcher',
    func = lambda query: search_content("lesson",query).to_dict(orient = "records"),
    description = "searches for relevant lessons or excercises in the database"
)

In [ ]:


# Initialize Agent 2 (Content Searcher) with the defined tool
agent_searcher = initialize_agent(
    tools=[tool_search],                      # list of tools this agent can use
    llm=llm,                                  # the underlying language model
    verbose=True                              # prints reasoning and steps
)

### Agent 3: Response Generator

This agent takes the retrieved content and composes an answer,  recommending the most relevan lesson and its link.

In [6]:
prompt_answer = PromptTemplate(

    input_variables = ["query","result"],
    template = """"
     You are an educational student. Based on the following search results, write a short friendly reply. 

     User  query: {query}

     Reuslts: {result}

     Compose a concise answer recommending the most suitable lesson and include the link.
     """
)


In [ ]:
agent_response = LLMChain(llm = llm,prompt=prompt_answer)

### Workflow coordination

Coordinate all three agents into a single functionL:
1. Query preprocessor classifies the input.
2. Content Searcher finds the matching material.
3. Response generator composes the final message.

In [ ]:
def workflow(query):

    category = processor_agent.run(query).strip().lower()
    print(f"Detected category: {category}")

    result = search_content(category,query)
    answer = agent_response.run(query=query, result = result.to_string())

    return answer

user_query = "what is the best lesson about Neural Networks"

final_answer = workflow(user_query)

print("Final system response: \n")
print(final_answer)